In [1]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import sys
sys.path.append('../../src')
import utils

2023-08-23 08:11:39.498546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 08:11:40.248149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
watches = pd.read_pickle('../../data/user-level-data')
videos = pd.read_pickle('../../data/videos_raw_metadata')

In [3]:
from collections import Counter


all_topics = []
all_tags = []

for video in videos:
    video = videos[video]
    
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    all_topics += topics
    
    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    all_tags += tags

all_topics = list(set(all_topics))

atc = Counter(all_tags)
atc = pd.DataFrame.from_dict(atc, 'index', columns=['count'])
atc = atc.sort_values(['count'], ascending=False).head(20000)
all_tags = list(atc.index)

def get_one_hot_vector(topics, tags):
    oh_topics = np.zeros(len(all_topics))
    for topic in topics:
        oh_topics[all_topics.index(topic)] = 1
        
    oh_tags = np.zeros(len(all_tags))
    for tag in tags:
        if tag in all_tags:
            oh_tags[all_tags.index(tag)] = 1
        
    # oh = np.concatenate([oh_topics , oh_tags])
    
    return oh_topics


def get_topic_vector(video):
    video = videos[video]
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    return get_one_hot_vector(topics, tags)


In [4]:
X = []
for video in tqdm(videos):
    X.append(get_topic_vector(video))

100%|██████████| 524537/524537 [20:20<00:00, 429.88it/s]


In [38]:
class Autoencoder(Model):
    def __init__(self, latent_dim, input_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
          layers.Input(shape=(input_dim,)),
          # layers.Dense(8192, activation='relu'),
          # layers.Dropout(0.2),
          layers.Embedding(input_dim, 2048),
          layers.Dropout(0.2),
          layers.Dense(latent_dim, activation='sigmoid'),
        ])
        
        self.decoder = tf.keras.Sequential([
          layers.Embedding(input_dim, 2048),
          layers.Dropout(0.2),
          layers.Dense(input_dim, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

tf.config.experimental_run_functions_eagerly(True)

EMB_SHAPE = 2048
INPUT_SHAPE = X[0].shape[0]

autoencoder = Autoencoder(EMB_SHAPE, INPUT_SHAPE)
autoencoder.compile(optimizer='adam', loss='cosine_similarity', metrics='cosine_similarity')

In [23]:
X = pkl.load(open('X-topic-encoder', 'rb'))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)

In [26]:
batch = 64
X_t = int(len(X_train)/batch)

In [30]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [36]:
def get_data(train):
   i = 0
   while(True):
      i += 1
      if i == X_t:
         i = 0
         np.random.shuffle(train)
      yield train[:batch], train[:batch]

In [39]:
autoencoder.fit(get_data(X_train), epochs=20, verbose=True, steps_per_epoch=X_t)

2023-08-23 09:00:31.277017: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 19.59GiB (rounded to 21036531712)requested by op Cast
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-23 09:00:31.277091: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-23 09:00:31.277105: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 336, Chunks in use: 336. 84.0KiB allocated for chunks. 84.0KiB in use in bin. 62.7KiB client-requested in use in bin.
2023-08-23 09:00:31.277112: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 1.0KiB client-requested in use in bin.
2023-08-23 09:00:31.277117: I tensorflow/tsl/framework/bf

ResourceExhaustedError: Exception encountered when calling layer 'embedding_1' (type Embedding).

{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Cast] name: 

Call arguments received by layer 'embedding_1' (type Embedding):
  • inputs=tf.Tensor(shape=(64, 20062, 4096), dtype=float32)

In [101]:
def get_data_test(train):
   i = 0
   xtt = int(len(train)/100)
   for i in range(xtt):
      yield train[:100], train[:100]

In [110]:
autoencoder.evaluate(get_data_test(X_test))

524/524 [==============================] - 5s 10ms/step - loss: -0.4717 - cosine_similarity: 0.4717


[-0.47171714901924133, 0.47171714901924133]

In [50]:
xt = X_test[:100]

In [49]:
autoencoder.evaluate()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                1284032   
                                                                 
Total params: 1284032 (4.90 MB)
Trainable params: 1284032 (4.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
autoencoder.evaluate(xt, xt)

4/4 [==============================] - 0s 9ms/step - loss: -0.4184 - cosine_similarity: 0.4184


/home/hsmhabib/.conda/envs/ytbase/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


[-0.41835975646972656, 0.41835975646972656]

In [30]:
autoencoder.save_weights('../../data/models/autoencoder-titles')

In [6]:
autoencoder = autoencoder.load_weights('../../data/models/autoencoder-titles')

In [29]:
autoencoder.evaluate(X_test, X_test)

3243/3243 [==============================] - 30s 9ms/step - loss: -0.9331 - cosine_similarity: 0.9331


[-0.9330801963806152, 0.9330801963806152]

In [35]:
embeddings = dict()

for video in tqdm(videos):
    emb = videos[video]
    emb = np.array([emb])
    emb = autoencoder.encoder(emb)
    embeddings[video] = emb

100%|██████████| 518735/518735 [49:56<00:00, 173.10it/s]


In [36]:
pkl.dump(embeddings, open('../../data/embeddings/title-autoencoded', 'wb'))